In [ ]:
from pathlib import Path

from training_raster_clipper.core.models import TrainingConfiguration, TrainingFunctions

from training_raster_clipper.core.logging import log_info

from training_raster_clipper.core.visualization import (
    plot_rgb_data_array,
    plot_array,
    plot_geodataframe,
)

from IPython.display import display

import matplotlib.pyplot as plt

from pathlib import Path

from rioxarray import open_rasterio
from geopandas import read_file
import geopandas as gpd
import numpy as np
import rioxarray
import xarray as xr
from affine import Affine
from geopandas.geodataframe import GeoDataFrame
from rasterio.features import rasterize
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from training_raster_clipper.custom_types import (
    BandNameType,
    ClassificationResult,
    ClassifiedSamples,
    FeatureClassNameToId,
    PolygonMask,
    ResolutionType,
)
import logging 

In [ ]:
# See https://stackoverflow.com/questions/18786912/get-output-from-the-logging-module-in-ipython-notebook
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("test") 

In [ ]:
raster_input_path = Path(
    "D:/Profils/rskandrani/Desktop/tutoriel S1/S2A_MSIL2A_20230823T104631_N0509_R051_T31TCJ_20230823T170355.SAFE"
)
polygons_input_path = Path(
    "D:/Profils/rskandrani/Desktop/tutoriel S1/polygons_rf.geojson"
)

config = TrainingConfiguration(
    verbose=True,
    show_plots=True,
    resolution=60,
    band_names=("B04", "B03", "B02", "B8A"),
    raster_input_path=raster_input_path,
    polygons_input_path=polygons_input_path,
    csv_output_path=(
        Path(".").resolve().parent / Path("generated/classified_points.csv")
    ),
    raster_output_path=(
        Path(".").resolve().parent / Path("generated/sklearn_raster.tiff")
    ),
    implementation_name="eschalk",
)
config

In [ ]:
verbose = config.verbose
show_plots = config.show_plots

resolution = config.resolution
band_names = config.band_names

raster_input_path = config.raster_input_path
polygons_input_path = config.polygons_input_path
csv_output_path = config.csv_output_path
raster_output_path = config.raster_output_path


### (1) Load a GeoJSON file with `geopandas`

In [ ]:
def load_feature_polygons(input_path: Path) -> GeoDataFrame:
    geodf = read_file(input_path)
    return GeoDataFrame(geodf).to_crs(32631)


In [ ]:
polygons = load_feature_polygons(polygons_input_path)
if verbose:
    log_info(polygons, "polygons")
if show_plots:
    plot_geodataframe(polygons, f"{load_feature_polygons.__name__}")


### (2) Load a Sentinel-2 raster with `rioxarray`

In [ ]:
def load_sentinel_data(
    sentinel_product_location: Path,
    resolution: ResolutionType,
    band_names: tuple[BandNameType, ...],
) -> xr.DataArray:
    """Loads sentinel product

    Example input path: `S2A_MSIL2A_20221116T105321_N0400_R051_T31TCJ_20221116T170958.SAFE`

    Args:
        sentinel_product_location (Path): Location of the .SAFE folder containing a Sentinel-2 product.

    Returns:
        xr.DataArray: A DataArray containing the 3 RGB bands from the visible spectrum
    """
    RADIO_ADD_OFFSET = -1000
    QUANTIFICATION_VALUE = 10000

    dict_bands = {}
    if len(band_names) == 0:
        return print("no bands selected")
    for band in band_names:
        dict_bands[band] = sorted(
            Path(sentinel_product_location).glob(
                f"GRANULE/*/IMG_DATA/R{resolution}m/*_{band}_*"
            )
        )[0]

    list_dataarray = []

    for band_name, raster_path in dict_bands.items():
        raster = open_rasterio(raster_path)

        list_dataarray.append(raster.assign_coords({"band": [band_name]}))

    rasters_concat = xr.concat(list_dataarray, dim="band")

    result = rasters_concat.where(
        rasters_concat != 0,
        np.float32(np.NaN),
    )

    result = (result + RADIO_ADD_OFFSET) / QUANTIFICATION_VALUE

    return result


In [ ]:
rasters = load_sentinel_data(raster_input_path, resolution, band_names)
if verbose:
    log_info(rasters, "rasters")
if show_plots:
    plot_rgb_data_array(rasters, f"{load_sentinel_data.__name__}")


In [ ]:
rasters.rio.crs


In [ ]:
list(reversed(rasters.shape[:2]))


In [ ]:
display(polygons)


In [ ]:
polygons.shape


In [ ]:
# rasters.spatial_ref.GeoTransform.split(' ')

# transform_tuple = (float(x) for x in rasters.spatial_ref.GeoTransform.split(' ') )
# tuple(transform_tuple)


### (3) Rasterize the polygons

In [ ]:
def rasterize_geojson(
    data_array: xr.DataArray,
    training_classes: GeoDataFrame,
) -> tuple[PolygonMask, FeatureClassNameToId]:
    """Burns a set of vectorial polygons to a raster.

    See https://gis.stackexchange.com/questions/316626/rasterio-features-rasterize

    Args:
        data_array (xr.DataArray): The Sentinel raster, from which data is taken, such as the transform or the shape.
        training_classes (GeoDataFrame): The input set of classified multipolygons to burn

    Returns:
        xr.DataArray: A mask raster generated from the polygons, representing the same geographical region as the source dataarray param
                      0 where no polygon were found, and integers representing classes in order of occurence in the GeoDataFrame
    """

    transform_tuple = tuple(
        float(x) for x in data_array.spatial_ref.GeoTransform.split(" ")
    )
    fwd = Affine.from_gdal(*transform_tuple)

    shape = data_array.isel(band=0).shape

    dict_mapping_class = dict(zip(training_classes.color, training_classes.index + 1))

    geodf_list = list(zip(training_classes.geometry, training_classes.index + 1))

    polygon_mask = rasterize(geodf_list, out_shape=shape, transform=fwd)

    return polygon_mask, dict_mapping_class

In [ ]:
burnt_polygons, mapping = rasterize_geojson(rasters, polygons)
if verbose:
    log_info(burnt_polygons, "burnt_polygons")
    log_info(mapping, "mapping")
if show_plots:
    plot_array(burnt_polygons, f"{rasterize_geojson.__name__}")


In [ ]:
burnt_polygons


### (4) Intersect the Sentinel-2 raster with polygons

In [ ]:
def produce_clips(
    data_array: xr.DataArray, burnt_polygons: PolygonMask, mapping: FeatureClassNameToId
) -> ClassifiedSamples:
    """Extract RGB values covered by classified polygons

    Args:
        data_array (xr.DataArray): RGB raster
        burnt_polygons (PolygonMask): Rasterized classified multipolygons

    Returns:
        _type_: A list of the RGB values contained in the data_array and their corresponding classes
    """
    polygons_data_array = data_array.isel(band=0).copy(data=burnt_polygons)

    dict_data_array_stacked = {
        "class": polygons_data_array.stack(z=("y", "x")),
        "reflectance": data_array.stack(z=("y", "x")),
    }

    classified_reflectances = xr.Dataset(dict_data_array_stacked)
    class_not_zero = classified_reflectances["class"] != 0
    classified_reflectances = classified_reflectances.isel(z=class_not_zero)

    return classified_reflectances

In [ ]:
classified_rgb_rows = produce_clips(rasters, burnt_polygons, mapping)
if verbose:
    log_info(classified_rgb_rows, "classified_rgb_rows")


In [ ]:
classified_rgb_rows.drop_vars(["x", "y", "spatial_ref"])


### (5) Persist the intersection to a CSV

In [ ]:
def persist_to_csv(
    classified_rgb_rows: ClassifiedSamples,
    csv_output_path: Path,
) -> None:
    df_classified_rgb_rows = classified_rgb_rows.to_dataframe().to_csv(
        csv_output_path + "/classified_rgb_rows.csv",
    )

In [ ]:
persist_to_csv(classified_rgb_rows, csv_output_path)
log_info(f"Written CSV output {csv_output_path}")


In [ ]:
classified_rgb_rows["reflectance"].values


In [ ]:
rasters.stack(z=("y", "x")).values


In [ ]:
classified_rgb_rows["class"].values


### (6) Train a machine learning model

In [ ]:
def classify_sentinel_data(
    rasters: xr.DataArray, classified_rgb_rows: ClassifiedSamples
) -> ClassificationResult:
    X_train = classified_rgb_rows["reflectance"].values
    y_train = classified_rgb_rows["class"].values

    model_rf = RandomForestClassifier().fit(X_train.T, y_train)

    predicted_classes = model_rf.predict(rasters.stack(z=("y", "x")).values.T)

    ref_raster = rasters.isel(band=0, drop=True)

    predicted_classes_xarray = ref_raster.copy(
        data=np.reshape(predicted_classes, ref_raster.shape)
    )

    return predicted_classes_xarray

In [ ]:
mapping

In [ ]:
classification_result = classify_sentinel_data(rasters, classified_rgb_rows)
if verbose:
    log_info(classification_result, "classification_result")
if show_plots:
    plot_array(classification_result, f"{classify_sentinel_data.__name__}")


### (7) Export the classification raster result

In [ ]:
def persist_classification_to_raster(
    raster_output_path: Path, classification_result: ClassificationResult
) -> None:
    classification_result.rio.to_raster(raster_output_path)


In [ ]:
persist_classification_to_raster(raster_output_path, classification_result)
log_info(f"Written Classified Raster to {csv_output_path}")

# --

log_info("Congratulations, you reached the end of the tutorial!")
